In [15]:
import pandas as pd
import numpy as np
import pickle
import os, time
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [16]:
# Reading CSV files, and merging all of them into a single DataFrame
CISIDS2017_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CICIDS2017ML"
CTU13_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CTU13ML"

CICIDS2017_df_list = []
CTU13_df_list = []

# Reading CICIDS2017 CSV files into a single DataFrame
for f in os.listdir(CISIDS2017_folder):
    file_path = os.path.join(CISIDS2017_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CICIDS2017_df_list.append(pd.read_csv(file_path))

# Reading CTU13 CSV files into a single DataFrame
for f in os.listdir(CTU13_folder):
    file_path = os.path.join(CTU13_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CTU13_df_list.append(pd.read_csv(file_path))

CICIDS2017_df, CTU13_df = pd.concat(CICIDS2017_df_list, ignore_index=True), pd.concat(CTU13_df_list, ignore_index=True)

Reading:  Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Morning.pcap_ISCX_Relabeled.csv
Reading:  Wednesday-workingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX_Relabeled.csv
Reading:  Monday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_Relabeled.csv
Reading:  Tuesday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX_Relabeled.csv
Reading:  CTU13_Normal_Traffic_Relabeled.csv
Reading:  CTU13_Attack_Traffic_Relabeled.csv


In [17]:
# QUICK PREPROCESSING. 
# Some classifiers do not like "infinite" (inf) or "null" (NaN) values.
CICIDS2017_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CICIDS2017_df.columns[CICIDS2017_df.isna().any()]))
CICIDS2017_df.dropna(inplace=True)

CTU13_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CTU13_df.columns[CTU13_df.isna().any()]))
CTU13_df.dropna(inplace=True)

Columns with problematic values:  [' Flow Packets/s']
Columns with problematic values:  []


In [18]:
CICIDS2017_df[' Label'].unique()

array(['BENIGN', 'PortScan', 'Bot', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'DDoS', 'FTP-Patator', 'SSH-Patator',
       'Infiltration'], dtype=object)

In [19]:
CTU13_df[' Label'].unique()

array(['BENIGN', 'Bot'], dtype=object)

In [20]:
# Create a new column that unifies all malicious classes into a single class for binary classification
CICIDS2017_df['GT'] = np.where(CICIDS2017_df[' Label']=='BENIGN', 'Benign', 'Malicious')
CTU13_df['GT'] = np.where(CTU13_df[' Label']=='BENIGN', 'Benign', 'Malicious')

In [21]:
features = pd.Index([
    ' Flow Duration',
    ' Total Fwd Packets',
    ' Total Backward Packets',
    ' Total Length of Bwd Packets',
    ' Fwd Packet Length Max', 
    ' Fwd Packet Length Min',
    ' Fwd Packet Length Mean',
    ' Fwd Packet Length Std',
    ' Bwd Packet Length Min',
    ' Bwd Packet Length Mean',
    ' Bwd Packet Length Std',
    ' Flow Packets/s',
    ' Flow IAT Mean',
    ' Flow IAT Std',
    ' Flow IAT Max',  
    ' Flow IAT Min',
    ' Fwd IAT Mean',
    ' Fwd IAT Std',
    ' Fwd IAT Max',
    ' Fwd IAT Min',
    ' Bwd IAT Mean',
    ' Bwd IAT Std',
    ' Bwd IAT Max',
    ' Bwd IAT Min',
    ' Bwd PSH Flags',
    ' Fwd Header Length',
    ' Bwd Header Length',
    ' Bwd Packets/s',
    ' Min Packet Length',
    ' Max Packet Length',
    ' Packet Length Mean',
    ' Packet Length Std', 
    ' Packet Length Variance',
    ' SYN Flag Count',
    ' RST Flag Count',
    ' ACK Flag Count',
    ' Down/Up Ratio',
    ' Average Packet Size', 
    ' Avg Fwd Segment Size',
    ' Avg Bwd Segment Size',
    ' Init_Win_bytes_backward',
    ' act_data_pkt_fwd',
    ' Active Std',
    ' Active Max',
    ' Active Min', 
    ' Idle Std',
    ' Idle Max',
    ' Idle Min' 
])

In [22]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Train a binary Dummy Classifier model on CTU13 dataset
start = time.time()
dummy_bin_ctu13 = DummyClassifier(strategy='stratified')
dummy_bin_ctu13.fit(train_ctu13[features], train_ctu13['GT'])
end = time.time() - start
print("Training time (CTU13 Binary): ", end)

# Save the binary Dummy Classifier model trained on CTU13 dataset
with open('../Pickle Files/DummyBinaryCTU13.pkl', 'wb') as file:
    pickle.dump(dummy_bin_ctu13, file)

Training time (CTU13 Binary):  0.07322359085083008


In [23]:
# Train test splits for both datasets 
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Train a binary Dummy Classifier model on CICIDS2017 dataset
start = time.time()
dummy_bin_cicids = DummyClassifier(strategy='stratified')
dummy_bin_cicids.fit(train_cicids[features], train_cicids['GT'])
end = time.time() - start
print("Training time (CICIDS2017 Binary): ", end)

# Save the binary Dummy Classifier model trained on CICIDS2017 dataset
with open('../Pickle Files/DummyBinaryCICIDS2017.pkl', 'wb') as file:
    pickle.dump(dummy_bin_cicids, file)

Training time (CICIDS2017 Binary):  4.4926745891571045


In [24]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Train a multiclass Dummy Classifier model on CICIDS2017 dataset
start = time.time()
dummy_multi_cicids = DummyClassifier(strategy='stratified')
dummy_multi_cicids.fit(train_cicids[features], train_cicids[' Label'])
end = time.time() - start
print("Training time (CICIDS2017 Multiclass): ", end)

# Save the multiclass Dummy Classifier model trained on CICIDS2017 dataset
with open('../Pickle Files/DummyMulticlassCICIDS2017.pkl', 'wb') as file:
    pickle.dump(dummy_multi_cicids, file)

Training time (CICIDS2017 Multiclass):  1.6379647254943848


In [25]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT']) 
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the binary Dummy Classifier model trained on CTU13 dataset
with open('../Pickle Files/DummyBinaryCTU13.pkl', 'rb') as file:
    dummy_bin_ctu13 = pickle.load(file)
    
# Test on CTU13 dataset
predictions_bin_ctu13 = dummy_bin_ctu13.predict(test_ctu13[features])
print("Acc (CTU13 Binary Dummy): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin_ctu13)))
print("F1-score (CTU13 Binary Dummy): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin_ctu13, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin_ctu13, rownames=['True'], colnames=['Pred'])

Acc (CTU13 Binary Dummy): 0.513244
F1-score (CTU13 Binary Dummy): 0.422144


Pred,Benign,Malicious
True,,
Benign,12373,8953
Malicious,9001,6558


In [26]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the binary Dummy Classifier model trained on CICIDS2017 dataset
with open('../Pickle Files/DummyBinaryCICIDS2017.pkl', 'rb') as file:
    dummy_bin_cicids = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_bin_cicids = dummy_bin_cicids.predict(test_cicids[features]) 
print("Acc (CICIDS2017 Binary Dummy): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("F1-score (CICIDS2017 Binary Dummy): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary Dummy): 0.684375
F1-score (CICIDS2017 Binary Dummy): 0.197910


Pred,Benign,Malicious
True,,
Benign,730086,178442
Malicious,178577,44046


In [27]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the binary Dummy Classifier model trained on CTU13 dataset
with open('../Pickle Files/DummyBinaryCTU13.pkl', 'rb') as file:
    dummy_bin_ctu13 = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_bin_cicids = dummy_bin_ctu13.predict(test_cicids[features])
print("Acc (CTU13 to CICIDS2017 Dummy): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids))) 
print("F1-score (CTU13 to CICIDS2017 Dummy): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CTU13 to CICIDS2017 Dummy): 0.547701
F1-score (CTU13 to CICIDS2017 Dummy): 0.268377


Pred,Benign,Malicious
True,,
Benign,525695,382833
Malicious,128786,93837


In [28]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the binary Dummy Classifier model trained on CICIDS2017 dataset 
with open('../Pickle Files/DummyBinaryCICIDS2017.pkl', 'rb') as file:
    dummy_bin_cicids = pickle.load(file)

# Test on CTU13 dataset
predictions_bin = dummy_bin_cicids.predict(test_ctu13[features])
print("Acc (CICIDS2017 Binary to CTU13 Dummy): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin)))
print("F1-score (CICIDS2017 Binary to CTU13 Dummy): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary to CTU13 Dummy): 0.547946
F1-score (CICIDS2017 Binary to CTU13 Dummy): 0.270923


Pred,Benign,Malicious
True,,
Benign,17113,4213
Malicious,12461,3098


In [29]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the multiclass Dummy Classifier model trained on CICIDS2017 dataset
with open('../Pickle Files/DummyMulticlassCICIDS2017.pkl', 'rb') as file:
    dummy_multi_cicids = pickle.load(file)

# Test on CTU13 dataset
predictions_multi = dummy_multi_cicids.predict(test_ctu13[features])
print("Acc (CICIDS2017 Multi-class to CTU13 Dummy): {:3f}".format(accuracy_score(test_ctu13[' Label'], predictions_multi)))
print("F1-score (CICIDS2017 Multi-class to CTU13 Dummy): {:3f}".format(f1_score(test_ctu13[' Label'], predictions_multi, average='macro')))
pd.crosstab(test_ctu13[' Label'], predictions_multi, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Multi-class to CTU13 Dummy): 0.463332
F1-score (CICIDS2017 Multi-class to CTU13 Dummy): 0.051769


Pred,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Infiltration,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � XSS
True,,,,,,,,,,,,,
BENIGN,17079,20,1016,78,1785,40,53,64,1,1130,44,12,4
Bot,12457,11,712,64,1246,26,38,45,1,906,45,6,2
